In [1]:
import pandas as pd
import time

In [2]:
file_start_time = time.time()

In [3]:
header = ['_id', 'type', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time', 'imo', 'callsign', 'shipname', 'shiptype', 'to_bow',
          'to_stern', 'to_port', 'to_starboard', 'month', 'day', 'hour', 'minute', 'draught', 'destination']
        # tbc header name from country_destination

In [4]:
start_time = time.time()

chunk = pd.read_csv('trial.t.csv',chunksize=100000,lineterminator='\n', names = header)
df = pd.concat(chunk)

# start_time = time.time()
end_time = time.time()
read_time =   end_time - start_time
print("Read time:",read_time)


Read time: 167.14272379875183


In [8]:
df.time.tail()

27171944    2023-09-30T23:59:55Z
27171945    2023-09-30T23:59:55Z
27171946    2023-09-30T23:59:55Z
27171947    2023-09-30T23:59:55Z
27171948    2023-09-30T23:59:55Z
Name: time, dtype: object

In [9]:
start_time = time.time()
df.isna().sum()
end_time = time.time()
execution_time =   end_time - start_time
print("Execution time:",execution_time)

Execution time: 9.841086626052856


In [10]:
df.shape

(27171949, 26)

# Cleaning steps
1. Drop message id col
2. Split by msg type (123, 5, 1819)
3. Remove MMSI less than 9 digits
4. Extract SG destination
   
For 123 and 1819:
1. convert status to int

For 5:

# NA
1. Callsign drop non alpha-numeric (callsign ~ '^[A-Za-z0-9]+$'  and > 0)
2. Convert shiptype to int
4. Drop month, day, hour minute


In [11]:
clean_start_time = time.time()

In [12]:
df_clean = df.drop('_id', axis = 1)

In [14]:
df_clean.shape

(27171949, 25)

In [15]:
# clean mmsi shorter than 9 digits
df_clean = df_clean[df_clean.mmsi > 99999999]

In [16]:
df_clean.shape

(27013326, 25)

In [17]:
start_time = time.time()
df_type123 = df_clean[df_clean.type <= 3][['mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time']]
print(df_type123.shape)
end_time = time.time()
execution_time =   end_time - start_time
print("Execution time:",execution_time)

(23223013, 10)
Execution time: 25.575968980789185


In [14]:
# start_time = time.time()
# df_type5 = df_clean[df_clean.type == 5][['mmsi', 'time', 'draught', 'country_destination']] # cleaned destination 
# print(df_type5.shape)
# end_time = time.time()
# execution_time =   end_time - start_time
# print("Execution time:",execution_time)

In [18]:
start_time = time.time()
df_type1819 = df_clean[df_clean.type >= 18][['mmsi', 'speed', 'accuracy', 'lon', 'lat', 'course', 'time']]
print(df_type1819.shape)
end_time = time.time()
execution_time =   end_time - start_time
print("Execution time:",execution_time)

(2373177, 7)
Execution time: 3.190220832824707


# Clean df_type 123

In [19]:
# change status to type int
df_type123['status'] = df_type123['status'].astype(int)

In [20]:
df_type123.shape

(23223013, 10)

In [21]:
df_type123.time.tail()

27171943    2023-09-30T23:59:55Z
27171945    2023-09-30T23:59:55Z
27171946    2023-09-30T23:59:55Z
27171947    2023-09-30T23:59:55Z
27171948    2023-09-30T23:59:55Z
Name: time, dtype: object

In [25]:
df_type123.dtypes

mmsi          int64
status        int32
turn        float64
speed       float64
accuracy    float64
lon         float64
lat         float64
course      float64
heading     float64
time         object
dtype: object

# Clean df_type 5

# Clean df_type 1819

In [22]:
clean_end_time = time.time()
clean_time =   clean_end_time - clean_start_time
print("Cleaning time:",clean_time)

Cleaning time: 118.38731908798218


# Destination cleaning (NA)

In [18]:
# # Singapore location cleaning
# keywords = ['sg', 'singapore', 'sin', 'sgp', 'PEBGA', 'PEBGB', 'PSBG', 'PWBGA', 'PWBGB', 'PJSB', 'PGBG', 'ACBTH', 'ACGP', 'AEBA', 'AEBB', 'AEBC', 'AEPB', 'AEPBB', 'AEPBC', 'AESPA', 'ASCA', 'ASCB', 'AMOW', 'AEW', 'AEPA', 'AEHA', 'AEHC', 'AEHB']

# # Function to clean destination data using input
# def clean_destination(destination):
#     destination = str(destination)
#     cleaned_destination = [word.upper() for word in keywords if word.lower() in destination.lower()]
#     if cleaned_destination:
#         return 'SG'
        
# # Apply the function to the 'destination' column and create a new column 'destination_clean'
# df_clean['destination_clean'] = df_clean['destination'].apply(clean_destination)

# # drop destination column
# # df_clean = df_clean.drop('destination', axis = 1)

In [19]:
# df_clean.destination_clean.value_counts()

In [20]:
# countries_keywords = {
#     'SG': ['sg', 'singapore', 'sin', 'sgp'],  # Singapore
#     'US': ['us', 'usa', 'america'],  # United States
#     'CN': ['cn', 'china', 'prc'],  # China
#     'JP': ['jp', 'japan'],  # Japan
#     'KR': ['kr', 'south korea'],  # South Korea
#     'DE': ['de', 'germany'],  # Germany
#     'NL': ['nl', 'netherlands'],  # Netherlands
#     'AE': ['ae', 'united arab emirates', 'uae'],  # United Arab Emirates
#     'HK': ['hk', 'hong kong'],  # Hong Kong
#     'MY': ['my', 'malaysia'],  # Malaysia
#     'BR': ['br', 'brazil'],  # Brazil
#     'AU': ['au', 'australia'],  # Australia
#     'ID': ['id', 'indonesia'],  # Indonesia
#     'TH': ['th', 'thailand'],  # Thailand
#     'VN': ['vn', 'vietnam'],  # Vietnam
#     'PH': ['ph', 'philippines'],  # Philippines
#     'MM': ['mm', 'myanmar'],  # Myanmar
#     'KH': ['kh', 'cambodia'],  # Cambodia
#     'LA': ['la', 'lao', 'laos'],  # Laos
#     'BN': ['bn', 'brunei'],  # Brunei
#     'TL': ['tl', 'timor-leste'],  # Timor-Leste
#     'IN': ['in', 'india'],  # India
#     'PK': ['pk', 'pakistan'],  # Pakistan
#     'BD': ['bd', 'bangladesh'],  # Bangladesh
#     'LK': ['lk', 'sri lanka'],  # Sri Lanka
#     'NP': ['np', 'nepal'],  # Nepal
#     'MM': ['mm', 'myanmar'],  # Myanmar
#     # Add more countries and their keywords as needed
# }

In [21]:
# start_time = time.time()

# for code, keywords in countries_keywords.items():
#     def clean_destination(destination):
#         destination = str(destination)
#         cleaned_destination = [word.upper() for word in keywords if word.lower() in destination.lower()]
#         if cleaned_destination:
#             return code

#     df_clean['destination_clean'] = df_clean['destination'].apply(clean_destination)

# end_time = time.time()
# execution_time =   end_time - start_time
# print("Execution time:",execution_time)
# df_clean.destination_clean.value_counts()

In [22]:
# def clean_destination(row):
#     destination = str(row['destination'].lower())
#     for code, keywords in countries_keywords.items():
#         if any(keyword in destination for keyword in keywords):
#             return code.upper()
#     return None

In [23]:
# # Apply the clean_destination function to create 'destination_clean' column
# df_clean['destination_clean'] = df_clean.apply(clean_destination, axis=1)

In [24]:
# df_clean.destination_clean.value_counts()

# All cleaning steps completed, save to csv

In [25]:
# df_clean.to_csv('df_clean.csv')

In [23]:
df_type123.to_csv('df_type123.csv')

In [30]:
df_type123.time

,mmsi,status,turn,speed,accuracy,lon,lat,course,heading,time
0,357007000,0,0.0,8.7,1.0,103.71341,1.15643,299.3,295.0,2023-09-01T00:00:00Z
1,477058300,1,0.0,0.0,1.0,103.56141,1.26156,287.0,100.0,2023-09-01T00:00:23Z
2,563542000,8,0.0,6.5,0.0,103.75856,1.26569,292.4,292.0,2023-09-01T00:00:23Z
3,563185100,0,0.0,0.0,0.0,103.76013,1.29132,261.6,189.0,2023-09-01T00:00:23Z
4,373943000,0,0.0,11.8,1.0,103.71131,1.07439,53.8,58.0,2023-09-01T00:00:45Z


In [27]:
# df_type5.to_csv('df_type5.csv')

In [28]:
df_type1819.to_csv('df_type1819.csv')

In [26]:
df_type1819.tail()

,mmsi,speed,accuracy,lon,lat,course,time
27171915,563023730,0.1,1.0,103.74246,1.29788,0.0,2023-09-30T23:59:53Z
27171920,563075660,0.1,1.0,103.75532,1.29455,58.3,2023-09-30T23:59:53Z
27171923,563013440,0.0,1.0,103.68815,1.25244,0.0,2023-09-30T23:59:54Z
27171935,542437110,3.8,1.0,103.75992,1.29340,87.8,2023-09-30T23:59:54Z
27171938,563021740,5.9,1.0,103.73118,1.29542,146.4,2023-09-30T23:59:54Z


In [29]:
file_end_time = time.time()
file_execution_time = file_end_time - file_start_time
print("Read time:",read_time)
print("Cleaning time:",clean_time)
print("File run time:",file_execution_time)

Read time: 59.438124656677246
Cleaning time: 34.3142032623291
File run time: 188.96060848236084
